<a href="https://colab.research.google.com/github/fralfaro/MAT281/blob/main/docs/labs/lab_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAT281 - Laboratorio N°04


**Objetivo**: Aplicar técnicas intermedias y avanzadas de análisis de datos con pandas utilizando un caso real: el Índice de Libertad de Prensa. Este laboratorio incluye operaciones de limpieza, transformación, combinación de datos, y análisis exploratorio usando `merge`, `groupby`, `concat` y otras funciones fundamentales.




**Descripción del Dataset**

El presente conjunto de datos está orientado al análisis del **Índice de Libertad de Prensa**, una métrica internacional que evalúa el nivel de libertad del que gozan periodistas y medios de comunicación en distintos países. Este índice es recopilado anualmente por la organización **Reporteros sin Fronteras**.

La base de datos contempla observaciones por país y año, e incluye tanto el valor del índice como el ranking correspondiente. A menor puntaje en el índice, mayor nivel de libertad de prensa.

**Diccionario de variables**

| Variable     | Clase    | Descripción                                                                          |
| ------------ | -------- | ------------------------------------------------------------------------------------ |
| `codigo_iso` | carácter | Código ISO 3166-1 alfa-3 que representa a cada país.                                 |
| `pais`       | carácter | Nombre oficial del país.                                                             |
| `anio`       | entero   | Año en que se registró la medición del índice.                                       |
| `indice`     | numérico | Valor numérico del Índice de Libertad de Prensa (menor valor indica mayor libertad). |
| `ranking`    | entero   | Posición relativa del país en el ranking mundial de libertad de prensa.              |


**Fuente original y adaptación pedagógica**

* **Fuente original**: [Reporteros sin Fronteras](https://www.rsf-es.org/), recopilado y publicado a través del portal del [Banco Mundial](https://tcdata360.worldbank.org/indicators/h3f86901f?country=BRA&indicator=32416&viz=line_chart&years=2001,2019).
* **Adaptación educativa**: Los archivos han sido modificados intencionalmente para incorporar desafíos técnicos que permiten aplicar los contenidos abordados en clases, tales como limpieza de datos, normalización, detección de duplicados, y combinación de fuentes.


**Descripción de los archivos disponibles**

* **`libertad_prensa_codigo.csv`**: Contiene los pares `codigo_iso` y `pais`. Incluye intencionalmente un código ISO con dos nombres distintos de país para efectos de limpieza y validación de datos.

* **`libertad_prensa_01.csv`**: Contiene registros de los años **anteriores a 2010**. Incluye las variables `PAIS`, `ANIO`, `INDICE`, y `RANKING` con nombres de columna en **mayúsculas**.

* **`libertad_prensa_02.csv`**: Contiene registros de los años **desde 2010 en adelante**. Estructura similar al archivo anterior, con nombres de columna también en **mayúsculas**.





In [62]:
import numpy as np
import pandas as pd

# lectura de datos
archivos_anio = [
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_01.csv',
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_02.csv'
 ]
df_codigos = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_codigo.csv')



### 1. Consolidación y limpieza de datos

A partir de los archivos disponibles, realice los siguientes pasos:

**a)** Cree un DataFrame llamado `df_anio` que consolide la información proveniente de los archivos **`libertad_prensa_01.csv`** y **`libertad_prensa_02.csv`**, correspondientes a distintas ventanas de tiempo. Recuerde que ambos archivos tienen nombres de columnas en mayúscula, por lo que debe normalizarlas a **minúscula** para asegurar consistencia.

**b)** Explore el archivo **`libertad_prensa_codigo.csv`** e identifique el código ISO que aparece asociado a dos nombres de país distintos. Elimine el registro que corresponda a un valor incorrecto o inconsistente, conservando solo el que considere válido.

**c)** Una vez preparados los archivos, cree un nuevo DataFrame llamado `df` que combine `df_anio` con `df_codigos`, utilizando la columna `codigo_iso` como clave. Asegúrese de realizar una unión que conserve únicamente los registros que tengan coincidencia en ambas fuentes.

> **Sugerencia**:
>
> * Para unir los archivos por filas (años), utilice la función `pd.concat([...])`.
> * Para combinar información por columnas (variables), utilice `pd.merge(...)` especificando `on='codigo_iso'`.



In [63]:
#a)
df = pd.DataFrame()
df1 = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_01.csv')
df2 = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_02.csv')

#normalizar columnas
df1.columns = df1.columns.str.lower()
df2.columns = df2.columns.str.lower()

df_anio = pd.concat((df1, df2))
df_anio.head()

,codigo_iso,anio,indice,ranking
0,AFG,2001,35.5,59.0
1,AGO,2001,30.2,50.0
2,ALB,2001,NaN,NaN
3,AND,2001,NaN,NaN
4,ARE,2001,NaN,NaN


In [64]:
#b)
df_codigos = df_codigos[df_codigos["pais"] != "malo"]
df_codigos = df_codigos[~((df_codigos["codigo_iso"] == "NER") & (df_codigos["pais"] != " Nigeria"))]

In [65]:
#c)
df = pd.merge(df_anio, df_codigos, on="codigo_iso", how="inner")
df.head()

,codigo_iso,anio,indice,ranking,pais
0,AFG,2001,35.5,59.0,Afghanistán
1,AGO,2001,30.2,50.0,Angola
2,ALB,2001,NaN,NaN,Albania
3,AND,2001,NaN,NaN,Andorra
4,ARE,2001,NaN,NaN,Emiratos Árabes Unidos




### 2. Exploración inicial del conjunto de datos

Una vez que hayas consolidado el DataFrame final `df`, realiza un análisis exploratorio básico respondiendo las siguientes preguntas:

#### **Estructura del DataFrame**

* ¿Cuántas **filas (observaciones)** contiene el conjunto de datos?
* ¿Cuántas **columnas** tiene el DataFrame?
* ¿Cuáles son los **nombres de las columnas**?
* ¿Qué **tipo de datos** tiene cada columna?
* ¿Hay columnas con un tipo de dato inesperado (por ejemplo, fechas como strings)?

#### **Resumen estadístico**

* Genera un resumen estadístico del conjunto de datos con `.describe()`.
  ¿Qué observas sobre los valores de `indice` y `ranking`?
* ¿Qué valores mínimo, máximo y promedio tiene la columna `indice`?
* ¿Qué países presentan los valores extremos en `indice` y `ranking`?

#### **Datos faltantes**

* ¿Cuántos valores nulos hay en cada columna?
* ¿Qué proporción de observaciones tienen valores faltantes?
* ¿Hay columnas con más del 30% de datos faltantes?

#### **Unicidad y duplicados**

* ¿Cuántos países distintos (`pais`) hay en el DataFrame?
* ¿Cuántos años distintos (`anio`) hay representados?
* ¿Existen filas duplicadas (exactamente iguales)? ¿Cuántas?

#### **Validación cruzada de columnas**

* ¿Hay inconsistencias entre el país (`pais`) y su código (`codigo_iso`)?
  (por ejemplo, un mismo código ISO asociado a más de un país)

> **Sugerencia**: Apoya tu análisis con funciones como `.info()`, `.nunique()`, `.isnull().sum()`, `.duplicated()`, `.value_counts()`, entre otras.



    

In [66]:
#estructura del dataframe
print(f"El dataframe tiene", df.shape, "filas y columnas respectivamente")
print(f"Los nombres de las columnas son: ", df.columns)
print(f"Los tipos de datos de cada columna son:\n", df.dtypes)
print(f"Si, país y codigo_iso son de tipo objeto y no string")

El dataframe tiene (3043, 5) filas y columnas respectivamente
Los nombres de las columnas son:  Index(['codigo_iso', 'anio', 'indice', 'ranking', 'pais'], dtype='object')
Los tipos de datos de cada columna son:
 codigo_iso     object
anio            int64
indice        float64
ranking       float64
pais           object
dtype: object
Si, país y codigo_iso son de tipo objeto y no string


In [67]:
#.describe
df.describe()

,anio,indice,ranking
count,3043.000000,2648.000000,2820.000000
mean,2009.941176,206.871412,480.533688
std,5.786029,2703.623037,6494.342299
min,2001.000000,0.000000,1.000000
25%,2005.000000,15.250000,34.750000
50%,2009.000000,28.045000,71.000000
75%,2015.000000,41.385000,110.000000
max,2019.000000,64536.000000,121056.000000


-En ambos, sus valores maximos son gigantes en comparacióna la media
-Minimo: 0, maximo: 64536 y promedio: 206


In [68]:
#maximos de indice y ranking
print("Para ranking: \n",df.loc[df["ranking"].idxmax(), ["pais", "ranking"]])
print("Para indice: \n",df.loc[df["indice"].idxmax(), ["pais", "indice"]])

Para ranking: 
 pais         Kosovo
ranking    121056.0
Name: 2237, dtype: object
Para indice: 
 pais       Kosovo
indice    64536.0
Name: 2058, dtype: object


In [69]:
#valores nulos
df.isnull().sum()

,0
codigo_iso,0
anio,0
indice,395
ranking,223
pais,0


In [70]:
#portencaje columnas que tienen al menos un valor null
df.isnull().mean()

,0
codigo_iso,0.000000
anio,0.000000
indice,0.129806
ranking,0.073283
pais,0.000000


No, no hay columnas con más de un 30% con valores NULL

In [71]:
#paises distintos
df["pais"].nunique()

179

In [72]:
#años distintos
df["anio"].nunique()

17

In [73]:
# Comprobar filas duplicadas
df.duplicated().sum()

np.int64(0)

No existen filas duplicadas, hay 0

In [74]:
df.duplicated(subset=["pais", "codigo_iso"]).sum()


np.int64(2864)

Si, hay 2864 filas duplicadas considerando pais y codigo_iso




### 3. Comparación regional: países latinoamericanos

En esta sección se busca identificar cuáles son los países de América Latina que han presentado los valores extremos del **Índice de Libertad de Prensa** en cada año observado.

> Recuerda que un menor puntaje en `indice` implica mayor libertad de prensa.

#### **Tareas:**

**a)** Utilizando un ciclo `for`, recorre cada año del conjunto de datos filtrado por países latinoamericanos, y determina para cada año:

* El país con el menor valor de `indice` (mayor libertad de prensa).
* El país con el mayor valor de `indice` (menor libertad de prensa).

**b)** Resuelve la misma tarea del punto anterior utilizando un enfoque vectorizado con `groupby`, sin usar ciclos explícitos.



#### **Lista de países latinoamericanos considerada:**

```python
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
           'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
           'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
           'USA', 'VEN']
```

> Puedes usar esta lista para filtrar el DataFrame final por la columna `codigo_iso`.



In [81]:
# respuesta
#a
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
       'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
       'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
       'USA', 'VEN']

df_latam = df[df["codigo_iso"].isin(america)].dropna(subset=["indice"])
resultados = []

for año in sorted(df_latam["anio"].unique()):
    df_año = df_latam[df_latam["anio"] == año]
    if df_año.empty:
        continue
    menor = df_año.loc[df_año["indice"].idxmin()]
    mayor = df_año.loc[df_año["indice"].idxmax()]
    resultados.append({
        "anio": año,
        "pais_menor_indice": menor["pais"],
        "indice_menor": menor["indice"],
        "pais_mayor_indice": mayor["pais"],
        "indice_mayor": mayor["indice"]
    })
df_america = pd.DataFrame(resultados)
print(df_america.head(10))

   anio  pais_menor_indice  indice_menor pais_mayor_indice  indice_mayor
0  2001             Canadá          0.80              Cuba         90.30
1  2002  Trinidad y Tobago          1.00              Cuba         97.83
2  2003  Trinidad y Tobago          2.00         Argentina      35826.00
3  2004  Trinidad y Tobago          2.00              Cuba         87.00
4  2005            Bolivia          4.50              Cuba         95.00
5  2006             Canadá          4.88              Cuba         96.17
6  2007             Canadá          3.33              Cuba         88.33
7  2008             Canadá          3.70              Cuba         94.00
8  2009     Estados Unidos          6.75              Cuba         78.00
9  2012            Jamaica          9.88              Cuba         71.64


In [83]:
#b
menor = df_latam.loc[df_latam.groupby("anio")["indice"].idxmin()].rename(columns={"pais":"pais_menor_indice","indice":"indice_menor"})
mayor = df_latam.loc[df_latam.groupby("anio")["indice"].idxmax()].rename(columns={"pais":"pais_mayor_indice","indice":"indice_mayor"})
df_america = menor[["anio","pais_menor_indice","indice_menor"]].merge(mayor[["anio","pais_mayor_indice","indice_mayor"]],on="anio")
print(df_america.head(10))


   anio  pais_menor_indice  indice_menor pais_mayor_indice  indice_mayor
0  2001             Canadá          0.80              Cuba         90.30
1  2002  Trinidad y Tobago          1.00              Cuba         97.83
2  2003  Trinidad y Tobago          2.00         Argentina      35826.00
3  2004  Trinidad y Tobago          2.00              Cuba         87.00
4  2005            Bolivia          4.50              Cuba         95.00
5  2006             Canadá          4.88              Cuba         96.17
6  2007             Canadá          3.33              Cuba         88.33
7  2008             Canadá          3.70              Cuba         94.00
8  2009     Estados Unidos          6.75              Cuba         78.00
9  2012            Jamaica          9.88              Cuba         71.64


### 4. Análisis anual del índice por país

En esta sección se busca analizar la evolución del **índice máximo** de libertad de prensa alcanzado por cada país a lo largo del tiempo.

#### **Tarea principal:**

* Construye una tabla dinámica (`pivot_table`) donde las **filas** correspondan a los países, las **columnas** a los años (`anio`) y los **valores** sean el `indice` máximo alcanzado por cada país en ese año.
* Asegúrate de reemplazar los valores nulos resultantes con `0`.

> **Hint**: Puedes utilizar el parámetro `fill_value=0` en `pd.pivot_table(...)`.



#### **Preguntas adicionales:**

**a)** ¿Qué país tiene el mayor valor de `indice` en toda la tabla resultante? ¿Y cuál tiene el menor (distinto de cero)?
**b)** ¿Qué años presentan en promedio los valores de `indice` más altos? ¿Y los más bajos?

> (Pista: usa `.mean(axis=0)` sobre la tabla pivot)

**c)** ¿Qué país muestra mayor **variabilidad** (diferencia entre su máximo y mínimo `indice` a lo largo del tiempo)?

> (Pista: aplica `.max(axis=1) - .min(axis=1)`)

**d)** ¿Existen países con índice constante a lo largo de todos los años registrados? ¿Cuáles?

**e)** ¿Qué países no tienen ningún dato (es decir, quedaron con todos los valores igual a 0)? ¿Podrías explicar por qué?





In [88]:
#tarea principal
df["indice"] = pd.to_numeric(df["indice"], errors='coerce')
pivot = pd.pivot_table(df,index="pais",columns="anio",values="indice",aggfunc="max",fill_value=0)
pivot.head()

anio,2001,2002,2003,2004,2005,2006,2007,2008,2009,2012,2013,2014,2015,2017,2018,2019
pais,,,,,,,,,,,,,,,,
Afghanistán,35.5,40.17,28.25,39.17,44.25,56.50,59.25,54.25,51.67,37.36,37.07,37.44,37.75,39.46,37.28,36.55
Albania,0.0,6.50,11.50,14.17,18.00,25.50,16.00,21.75,21.50,30.88,29.92,28.77,29.92,29.92,29.49,29.84
Alemania,1.5,1.33,2.00,4.00,5.50,5.75,4.50,3.50,4.25,10.24,10.23,11.47,14.80,14.97,14.39,14.60
Algeria,31.0,33.00,43.50,40.33,40.00,40.50,31.33,49.56,47.33,36.54,36.26,36.63,41.69,42.83,43.13,45.75
Andorra,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6.82,6.82,19.87,19.87,21.03,22.21,24.63


In [94]:
#a
pais_max = pivot.max(axis=1).idxmax()
valor_max = pivot.max(axis=1).max()
pivot_no0 = pivot.replace(0, pd.NA)
pais_min = pivot_no0.min(axis=1).idxmin()
valor_min = pivot_no0.min(axis=1).min()

print(f"Mayor índice:", pais_max, valor_max)
print(f"Menor índice distinto de 0:", pais_min, valor_min)

Mayor índice: Kosovo 64536.0
Menor índice distinto de 0: Austria 0.5


In [95]:
#b
promedios_anio = pivot.mean(axis=0)

año_max_prom = promedios_anio.idxmax()
año_min_prom = promedios_anio.idxmin()

print(f"Año con promedio más alto:", año_max_prom, promedios_anio[año_max_prom])
print(f"Año con promedio más bajo:", año_min_prom, promedios_anio[año_min_prom])

Año con promedio más alto: 2013 449.11446927374294
Año con promedio más bajo: 2001 20.015642458100558


In [96]:
#c
variabilidad = pivot.max(axis=1) - pivot.min(axis=1)
pais_mayor_var = variabilidad.idxmax()
valor_var = variabilidad.max()

print(f"País con mayor variabilidad:", pais_mayor_var, valor_var)

País con mayor variabilidad: Kosovo 64536.0


In [97]:
#d
constantes = pivot[(pivot.max(axis=1) - pivot.min(axis=1)) == 0].index.tolist()
print(f"Países con índice constante:", constantes)

Países con índice constante: []


In [98]:
#e
sin_datos = pivot[(pivot == 0).all(axis=1)].index.tolist()
print(f"Países sin datos:", sin_datos)

Países sin datos: []


Todos los paises del pivot tienen al menos un valor de indice distinto de 0, por lo que ninguna fila quedó totalmente en 0